<a href="https://colab.research.google.com/github/adithya70/multi-modal-with-flask-and-streamlit/blob/main/streamlit_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [11]:
%%writefile streamlit_app.py
import streamlit as st
import requests
from PIL import Image
import io

# Set your Flask API URL (Update with your actual Ngrok URL)
FLASK_API_URL = "https://8f87-34-139-144-212.ngrok-free.app//predict"

st.title("📸 Multi-Modal Image Captioning")
st.write("Upload an image and enter a caption to get a prediction.")

# Upload Image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
text_input = st.text_input("Enter a caption")

if uploaded_file and text_input:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Convert image to bytes for API request
    img_bytes = io.BytesIO()
    image.save(img_bytes, format="JPEG")
    img_bytes = img_bytes.getvalue()

    if st.button("Predict"):
        with st.spinner("Processing..."):
            # Send request to Flask API
            response = requests.post(
                FLASK_API_URL,
                files={"image": ("image.jpg", img_bytes, "image/jpeg")},
                data={"text": text_input}
            )

            if response.status_code == 200:
                result = response.json()
                st.success("Prediction received!")
                st.write("### 🔍 Model Output:")
                st.json(result)
            else:
                st.error("Failed to get a prediction. Please try again.")


Overwriting streamlit_app.py


In [6]:
!ngrok authtoken 2shFBySTClb5xE6aeZ8rpkjhu0A_7FuVC2vsaSKVCiv1Y3Vta

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Start a new tunnel for Streamlit
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit Public URL: {public_url}")

# Run Streamlit
!streamlit run streamlit_app.py &>/dev/null &


🚀 Streamlit Public URL: NgrokTunnel: "https://5ea6-34-83-84-93.ngrok-free.app" -> "http://localhost:8501"


In [10]:
!curl -X POST "https://8f87-34-139-144-212.ngrok-free.app/predict" \
-F "image=@/content/2098418613_85a0c9afea.jpg" \
-F "text=A dog is running in a field."


{"input_text":"A dog is running in a field.","message":"Prediction successful"}
